# Mini Project - Predicting UK Data Scientist Salaries

- Part 1: Web Scraping and Data Cleaning

### Business Case Overview

You're working as a data scientist for a contracting firm that's rapidly expanding. Now that they have their most valuable employee (you!), they need to leverage data to win more contracts. Your firm offers technology and scientific solutions and wants to be competitive in the hiring market. Your principal wants you to

   - determine the industry factors that are most important in predicting the salary amounts for these data.

To limit the scope, your principal has suggested that you *focus on data-related job postings*, e.g. data scientist, data analyst, research scientist, business intelligence, and any others you might think of. You may also want to decrease the scope by *limiting your search to a single region.*

Hint: Aggregators like [Indeed.com](https://www.indeed.com) regularly pool job postings from a variety of markets and industries.

**Goal:** Scrape your own data from a job aggregation tool like Indeed.com in order to collect the data to best answer this question.

---

### Directions

In this project you will be leveraging a variety of skills. The first will be to use the web-scraping and/or API techniques you've learned to collect data on data jobs from Indeed.com or another aggregator. Once you have collected and cleaned the data, you will use it to address the question above.

### Factors that impact salary

To predict salary the most appropriate approach would be a regression model.
Here instead we just want to estimate which factors (like location, job title, job level, industry sector) lead to high or low salary and work with a classification model. To do so, split the salary into two groups of high and low salary, for example by choosing the median salary as a threshold (in principle you could choose any single or multiple splitting points).

Use all the skills you have learned so far to build a predictive model.
Whatever you decide to use, the most important thing is to justify your choices and interpret your results. *Communication of your process is key.* Note that most listings **DO NOT** come with salary information. You'll need to be able to extrapolate or predict the expected salaries for these listings.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10").

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters:

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [1]:
# Change URL below:

# Target min 2000 jobs with salaries London & min 2000 each for rest of UK cities

URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [567]:
import requests
import urllib.request
import bs4
from bs4 import BeautifulSoup
import time
from tqdm import tqdm # a progress bar

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

### Test for multipage:

- edit range in for loop below:

In [4]:
# Test for multipage:

URL = "https://www.indeed.co.uk/jobs?q=Data+Scientist&l=London&rqf=1&start="

job_titles = []
companies = []
salaries = []
locations = []
ratings = []

# Preliminary test for 5 pages:

for page in tqdm(range(10, 60, 10)):
    
    time.sleep(3)
    
    r = requests.get(URL.format(page))
    soup = BeautifulSoup(r.text, 'html.parser')
    listings = soup.find_all('div', attrs={'class': 'jobsearch-SerpJobCard'})
    
    for listing in listings:
        
            try:
                job_titles.append(listing.find('a', target='_blank').text.strip())
            except:
                job_titles.append(np.nan)
        
            try:
                companies.append(listing.find('span', class_='company').text.strip())
            except:
                companies.append(np.nan)
        
            try:
                salaries.append(listing.find('div', class_='salarySnippet salarySnippetDemphasizeholisticSalary').text.strip())
            except:
                salaries.append(np.nan)
        
            try:
                locations.append(listing.find('span', class_='location accessible-contrast-color-location').text.strip())
            except:
                locations.append(np.nan)

            try:
                ratings.append(listing.find('span', class_='ratingsDisplay').text.strip())
            except:
                ratings.append(np.nan)

100%|██████████| 5/5 [00:23<00:00,  4.61s/it]


In [5]:
pd.Series(job_titles).value_counts()

Data Scientist                                        37
Junior Data Scientist                                  5
Junior Data Scientist Placement                        5
Anti-Fraud Bribery and Corruption Data Scientist       5
Graduate Program - Associate Data Scientist            5
Artificial Intelligence – Data Scientist               5
Higher Statistical Data Scientist - across the GSS     4
Product Data Scientist                                 4
dtype: int64

In [6]:
pd.Series(salaries).isnull().sum()

56

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is in a `span` with `class='salaryText'`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element='jobTitle'`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 
- Decide which other components could be relevant, for example the region or the summary of the job advert.

### Write 4 functions to extract each item: location, company, job, and salary.

Example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- **Make sure these functions are robust and can handle cases where the data/field may not be available.**
    - Remember to check if a field is empty or `None` for attempting to call methods on it.
    - Remember to use `try/except` if you anticipate errors.
- **Test** the functions on the results above and simple examples.

- Save them as a list of lists

### Change the URL slightly for better 'parameters'


Consider:
- Search term could widen to include 'Data Analyst', 'Data Engineer' or 'ML Engineer'
- Limit to full & permanent jobs
- Try to get pages with more declared salaries
- Edit area/radius range for London - not to small that it limits results, not too large in case of overlaps with other cities

In [8]:
# More precise URL format for London
# 50 results per page

URL_LDN = 'https://www.indeed.co.uk/jobs?q=%28Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer%29+£20%2C000%2B&l=London&radius=50&jt=fulltime&limit=50&start={}'
URL_LDN

'https://www.indeed.co.uk/jobs?q=%28Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer%29+£20%2C000%2B&l=London&radius=50&jt=fulltime&limit=50&start={}'

In [9]:
# Secondary URL format for NYC to test:
# Same format as London
# US site allows me to specify additional parameter for entry level jobs:

URL_NYC = 'https://www.indeed.com/jobs?q=%28Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer%29+%2430%2C000%2B&l=New+York+City&radius=50&jt=fulltime&explvl=entry_level&limit=50&start={}'
URL_NYC

'https://www.indeed.com/jobs?q=%28Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer%29+%2430%2C000%2B&l=New+York+City&radius=50&jt=fulltime&explvl=entry_level&limit=50&start={}'

### Job Titles

In [10]:
# Job title (singular - single use function specific to Indeed website structure):

def extract_job_title(listing):
    try:
        job_title = listing.find('a', attrs={'target': '_blank'}).text.strip()
    except:
        job_title = 'None'
    return job_title

In [11]:
# Full Job Titles function:
# Test formatting (only applicable to Indeed website structure):


job_titles_full = []


# 5 pages, 250 results default to test
# At this point, URL needs to be specific to one country only

def extract_all_jobtitles(URL, max_page=100):
        
        job_titles_full = []
        
        # edited search terms to list 50 results per page:
        for page in tqdm(range(0, max_page, 50)):
            
            # random seconds per iteration
            time.sleep(np.abs(np.random.normal(loc=3, scale=0.5)))
            
            r = requests.get(URL.format(page))
            soup = BeautifulSoup(r.text, 'html.parser')
            listings = soup.find_all('div', attrs={'class': 'jobsearch-SerpJobCard'})

            for listing in listings:
                
                job_titles_full.append(extract_job_title(listing))
    
        return job_titles_full 

In [12]:
# London Test

ldn_job_titles = extract_all_jobtitles(URL_LDN)
ldn_job_titles[:10]

100%|██████████| 2/2 [00:10<00:00,  5.50s/it]


['Data and Research Analyst',
 'Graduate Program - Associate Data Scientist',
 'Data Scientist',
 'Data Analyst – Content & Media',
 'Machine Learning Engineer',
 'Data Analyst',
 'NHS Test and Trace - Data Privacy Analyst',
 'Machine Learning Engineer - Search and Recommendations',
 'Data and Reporting Analyst',
 'Data Entry Operator(UW-9K8754)']

In [13]:
len(ldn_job_titles)

100

In [14]:
# NYC Test:

nyc_job_titles = extract_all_jobtitles(URL_NYC)
nyc_job_titles[:10]

100%|██████████| 2/2 [00:10<00:00,  5.16s/it]


['Data Scientist',
 'Data Scientist',
 'Data Analyst',
 'Healthcare Data Scientist',
 'Data Scientist',
 'Data Scientist',
 'Analyst, Data and Analysis',
 'Data Analyst',
 'Data Scientist',
 'Data Analyst']

In [15]:
len(nyc_job_titles)

100

### Company Names

In [16]:
# Company Names - Consider multi try/except:


def extract_company_name(listing):
    try:
        company = listing.find('span', attrs={'class': 'company'}).text.strip()
    except:
        try:
            company = listing.find('a', attrs={'data-tn-element': 'companyName', 
                                               'class': 'turnstileLink',
                                               'target': '_blank'}).text.strip()
        except:
            company = 'None'
    return company

In [17]:
# Full Company Names function:


def extract_all_companies(URL, max_page=100):
        
        companies_full = []
        
        # edited search terms to list 50 results per page:
        for page in tqdm(range(0, max_page, 50)):
            
            # random seconds per iteration
            time.sleep(np.abs(np.random.normal(loc=3, scale=0.5)))
            
            r = requests.get(URL.format(page))
            soup = BeautifulSoup(r.text, 'html.parser')
            listings = soup.find_all('div', attrs={'class': 'jobsearch-SerpJobCard'})

            for listing in listings:
                
                companies_full.append(extract_company_name(listing))
    
        return companies_full

In [18]:
# Test UK:

ldn_companies = extract_all_companies(URL_LDN)
ldn_companies[:10]

100%|██████████| 2/2 [00:12<00:00,  6.16s/it]


['Cancer Research UK',
 'RELX Group',
 'NHS Midlands and Lancashire Commissioning Support...',
 'TalentPool',
 'IG Group',
 'Central London Community Healthcare NHS Trust',
 'UK Government - Department of Health and Social Ca...',
 'Twitter',
 'Cancer Research UK',
 'Binzagr Group']

In [19]:
len(ldn_companies)

100

### Salaries

In [20]:
# Salaries - multi try/except:
# Numerical & target col - be more thorough here

def extract_salary(listing):
    try:
        salary = listing.find('div', attrs={'class': 'salarySnippet salarySnippetDemphasizeholisticSalary'}).text.strip()
    except:
        try:
            salary = listing.find('div', attrs={'class': 'salarySnippet holisticSalary'}).text.strip()
        except:
            try:
                salary = listing.find('span', attrs={'class': 'salary no-wrap'}).text.strip()
            except:
                try:
                    salary = listing.find('span', attrs={'class': 'salaryText'}).text.strip()
                except:
                    salary = np.nan
    return salary

In [21]:
# Full Salaries function:

def extract_all_salaries(URL, max_page=100):
        
        salaries_full = []
        
        # edited search terms to list 50 results per page:
        for page in tqdm(range(0, max_page, 50)):
            
            # random seconds per iteration
            time.sleep(np.abs(np.random.normal(loc=3, scale=0.5)))
            
            r = requests.get(URL.format(page))
            soup = BeautifulSoup(r.text, 'html.parser')
            listings = soup.find_all('div', attrs={'class': 'jobsearch-SerpJobCard'})

            for listing in listings:
                
                salaries_full.append(extract_salary(listing))
    
        return salaries_full

In [22]:
# Test NYC

nyc_salaries = extract_all_salaries(URL_NYC)
nyc_salaries[:10]

100%|██████████| 2/2 [00:11<00:00,  5.56s/it]


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]

In [23]:
# Not too good
# Salaries are in ranges and different units (e.g. some on hourly basis)

nyc_salaries = pd.Series(nyc_salaries)
nyc_salaries.value_counts()

$25 - $35 an hour           1
$32,520 - $42,191 a year    1
$18 an hour                 1
$54,100 - $62,215 a year    1
$64,140 - $77,609 a year    1
$54,100 - $83,981 a year    1
$32,260 - $39,028 a year    1
$35,000 - $55,000 a year    1
$65,000 a year              1
$35.24 an hour              1
$18.25 an hour              1
dtype: int64

### Salary Cleaning

- Turn all below to function that can be reused later

In [24]:
# still needs to convert tuple to int & average the lower/upper values:

def salary_cleaner(col):
    col = col.dropna()
    mask = col.str.contains("a year")
    col = col[mask]
    col = col.replace("""[^0-9\.-]""", '', regex=True)
    col = col.str.split('-')
    col = col.apply(lambda x: tuple(x) if isinstance(x, list) else x)
    
    
    return col


In [25]:
nyc_salaries_cln = salary_cleaner(nyc_salaries)
nyc_salaries_cln


16    (54100, 83981)
18    (54100, 62215)
38    (32520, 42191)
58    (64140, 77609)
80    (35000, 55000)
81    (32260, 39028)
82          (65000,)
dtype: object

In [26]:
# Convert to int and avg function ((lower + upper) / 2):

### Locations

- In full function - add city col to deal with this

In [27]:
# Locations - multi try/except:

def extract_location(listing):
    try:
        location = listing.find('span', attrs={'class': 'location accessible-contrast-color-location'}).text.strip()
    except:
        location = 'None'
        
    return location

In [28]:
# Full Locations function:

def extract_all_locations(URL, max_page=100):
        
        locations_full = []
        
        # edited search terms to list 50 results per page:
        for page in tqdm(range(0, max_page, 50)):
            
            # random seconds per iteration
            time.sleep(np.abs(np.random.normal(loc=3, scale=0.5)))
            
            r = requests.get(URL.format(page))
            soup = BeautifulSoup(r.text, 'html.parser')
            listings = soup.find_all('div', attrs={'class': 'jobsearch-SerpJobCard'})

            for listing in listings:
                
                locations_full.append(extract_location(listing))
    
        return locations_full

In [29]:
# Test (UK)
# Not all exactly London, will require cleaning

ldn_locations = extract_all_locations(URL_LDN)
ldn_locations[:10]

100%|██████████| 2/2 [00:12<00:00,  6.24s/it]


['Stratford',
 'London',
 'London SE1 6LH',
 'London EC1R 4RB',
 'London',
 'London SW1P 2PF',
 'London',
 'London',
 'Stratford',
 'London']

In [30]:
len(ldn_locations)

100

### Website Links (bonus)

In [31]:
# Website Links - multi try/except:

def extract_link(listing):
    try:
        tag = listing.find('a', href=True)
        link = tag.get('href')

    except:
        link = 'None'
        
    return link

In [32]:
# Full website links function:

def extract_all_links(URL, max_page=100):
        
        links_full = []
        
        # edited search terms to list 50 results per page:
        for page in tqdm(range(0, max_page, 50)):
            
            # random seconds per iteration
            time.sleep(np.abs(np.random.normal(loc=3, scale=0.5)))
            
            r = requests.get(URL.format(page))
            soup = BeautifulSoup(r.text, 'lxml')
            listings = soup.find_all('div', attrs={'class': 'jobsearch-SerpJobCard'})

            for listing in listings:
                
                links_full.append(extract_link(listing))
    
        return links_full


In [33]:
# Test(NYC):
# Unsure if I'd want it in this format

nyc_links = extract_all_links(URL_NYC, 100)
nyc_links[:10]

100%|██████████| 2/2 [00:10<00:00,  5.24s/it]


['/rc/clk?jk=d7db13cc30ffa29e&fccid=18d58239dbfc6097&vjs=3',
 '/rc/clk?jk=21035cf0ea38aabc&fccid=13278012efd32c2a&vjs=3',
 '/rc/clk?jk=d7060a555b8cc761&fccid=e3c82d82eecba317&vjs=3',
 '/rc/clk?jk=6720cf1c03a1c426&fccid=2c23f29fcd5c78da&vjs=3',
 '/rc/clk?jk=1636faa0308fd08f&fccid=1577085fc2290983&vjs=3',
 '/rc/clk?jk=d492b2efea2071c5&fccid=2187d710fd27cd06&vjs=3',
 '/rc/clk?jk=514bca6873f60673&fccid=274e15dee00b5237&vjs=3',
 '/rc/clk?jk=b9b558dad7ef068a&fccid=50d9950288add6cc&vjs=3',
 '/rc/clk?jk=40b4fe6499314552&fccid=647578535675f915&vjs=3',
 '/rc/clk?jk=48624136752ce056&fccid=4a1076c699e1befc&vjs=3']

In [34]:
len(nyc_links)

100

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

-  JOB TITLE, COMPANY, SALARY, LOCATION, LINK:

## Do with individual cities, save as function

- change radius to reduce overlaps (30 mi)

- alter city list (UK) to also reduce overlaps (cities 30+ miles apart)

In [41]:
def alljob_extractor(URL, city, max_pages):
    
    """Get a dataframe per city without duplicates.
       Set max_pages equal to no. of listings you're targeting.
       Salaries will have np.nan in them for you to
       review and drop later."""
    
    results = {'Job_Title': [], 'Company': [],
                  'Salary': [], 'Location': [],
                  'City': [], 'Link': []}  
    
    job_titles_UK = []
    companies_UK = []
    salaries_UK = []
    locations_UK = []
    cities_UK = []
    links_UK = []

    for page in tqdm(range(0, max_pages, 50)):
        
        # Grab the results from the request (as above)
        # Append to the full set of results
        
        # random seconds per iteration (IMPORTANT to avoid website timeout)
        # use higher loc in future
        time.sleep(np.abs(np.random.normal(loc=3, scale=0.5)))
            
        r = requests.get(URL.format(page))
        
        soup = BeautifulSoup(r.text, 'lxml')
        listings = soup.find_all('div', attrs={'class': 'jobsearch-SerpJobCard'})
        
        for listing in listings:
            
            try:
                job_titles_UK.append(extract_job_title(listing))
            except:
                job_titles_UK.append('None')
                
            try:
                companies_UK.append(extract_company_name(listing))
            except:
                companies_UK.append('None')
                
            try:
                salaries_UK.append(extract_salary(listing))
            except:
                salaries_UK.append(np.nan)
                
            try:
                locations_UK.append(extract_location(listing))
            except:
                locations_UK.append('None')
            

            try:
                cities_UK.append(city)
            except:
                cities_UK.append('None')
            
            try:
                links_UK.append(extract_link(listing))
            except:
                links_UK.append('None')
                
    results['Job_Title'] = job_titles_UK
    results['Company'] = companies_UK
    results['Salary'] = salaries_UK
    results['Location'] = locations_UK
    results['City'] = city
    results['Link'] = links_UK
    
    dfprelim = pd.DataFrame.from_dict(results)
    
    # by default duplicates removed if matched on all columns
    dfprelim = dfprelim.drop_duplicates()
    
    return dfprelim

### City 1: London

In [42]:
URLLON = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=London&radius=30&jt=fulltime&limit=50&start={}"

In [43]:
lon_testdf = alljob_extractor(URLLON, "London", max_pages=5000)
lon_testdf.head()

100%|██████████| 100/100 [09:17<00:00,  5.57s/it]


,Job_Title,Company,Salary,Location,City,Link
0,Data and Research Analyst,Cancer Research UK,"£31,000 a year",Stratford,London,/rc/clk?jk=16e8b8e96810af24&fccid=0d06dfb448b2...
1,Graduate Program - Associate Data Scientist,RELX Group,NaN,London,London,/rc/clk?jk=b6751e8c9cd1c813&fccid=06206ab329e0...
2,Data Scientist,NHS Midlands and Lancashire Commissioning Supp...,"£24,907 - £30,615 a year",London SE1 6LH,London,/rc/clk?jk=195753e0562ed694&fccid=37c85e352ebb...
3,Data Analyst – Content & Media,TalentPool,"£27,000 a year",London EC1R 4RB,London,/rc/clk?jk=8c1cf0cc8aa52ba9&fccid=3e983d79737f...
4,Machine Learning Engineer,IG Group,NaN,London,London,/rc/clk?jk=f2bb4bb3f97b89a1&fccid=29f542c4b680...


In [44]:
# tricky to extract non duplicate jobs from london with declared salaries

lon_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,1000,1000,377,1000,1000,1000
unique,892,573,269,208,1,1000
top,Data Scientist,Public Health England,"£45,780 - £51,393 a year",London,London,/rc/clk?jk=f6e4cea3be180dfc&fccid=a65be4ca39f5...
freq,15,10,7,671,1000,1


### City 2: Edinburgh

In [46]:
URLEDI = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=Edinburgh&radius=30&jt=fulltime&limit=50&start={}"

In [47]:
edi_testdf = alljob_extractor(URLEDI, "Edinburgh", max_pages=2500)

100%|██████████| 50/50 [04:20<00:00,  5.22s/it]


In [48]:
edi_testdf.head()

,Job_Title,Company,Salary,Location,City,Link
0,Machine Learning Engineer,SoulTek,"£50,000 - £60,000 a year",Edinburgh,Edinburgh,/rc/clk?jk=ffdad69b595560bd&fccid=dbf7c402233a...
1,Customer Support roles - IRC85965E,Scottish Government,"£20,064 - £22,232 a year",Edinburgh EH11,Edinburgh,/rc/clk?jk=9b75e802a019bcdd&fccid=7bbf7f016062...
2,Data Analyst,Sopra Steria,"£50,000 a year",Edinburgh EH4 2HS,Edinburgh,/rc/clk?jk=778419f66a40c220&fccid=3ba6d20cd884...
3,Learning and Development Administrator,Scottish Government,"£20,064 - £22,232 a year",Edinburgh,Edinburgh,/rc/clk?jk=999428b8d88179a5&fccid=7bbf7f016062...
4,Graduate Process Engineer - Carbon,Veolia,"£26,000 - £28,000 a year",Edinburgh,Edinburgh,/company/Veolia/jobs/Graduate-Process-Engineer...


In [49]:
edi_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,1022,1022,447,1022,1022,1022
unique,877,460,285,192,1,991
top,Senior Software Engineer,Appoint Group,"£25,000 - £30,000 a year",Edinburgh,Edinburgh,/rc/clk?jk=44e74c1e8eae8cac&fccid=3eb4d72008bb...
freq,8,27,9,545,1022,2


### City 3: Manchester

In [50]:
URLMCR = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=Manchester&radius=30&jt=fulltime&limit=50&start={}"

In [51]:
mcr_testdf = alljob_extractor(URLMCR, "Manchester", max_pages=2500)

100%|██████████| 50/50 [04:28<00:00,  5.36s/it]


In [52]:
mcr_testdf.head()

,Job_Title,Company,Salary,Location,City,Link
0,Sustainability Data and Systems Analyst,Manchester University NHS Foundation Trust,"£24,907 - £30,615 a year",Wythenshawe,Manchester,/rc/clk?jk=db6d1d647ba993e6&fccid=9e0482cdf064...
1,Data Analyst,Salford Royal NHS Foundation Trust,"£38,890 - £44,503 a year",Manchester,Manchester,/rc/clk?jk=ddd62afe8f1bb057&fccid=e81f055df2aa...
2,Healthcare Scientist Practitoner,Public Health England,"£23,139 - £28,195 a year",Manchester M13 9WL,Manchester,/rc/clk?jk=8b5e3bd0af8e45a8&fccid=56df6d0891c9...
3,SSNAP Data Collector,Salford Royal NHS Foundation Trust,"£21,892 - £24,157 a year",Salford M6,Manchester,/rc/clk?jk=fd37c2b150b97a07&fccid=e81f055df2aa...
4,Learning Technologist,The Christie NHS Foundation Trust,"£24,907 - £30,615 a year",Manchester M20 4BX,Manchester,/rc/clk?jk=aa9039a181d8d803&fccid=4c197c46f279...


In [53]:
mcr_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,998,998,502,998,998,998
unique,926,446,276,209,1,998
top,Business Analyst,Jacobs,"£24,907 - £30,615 a year",Manchester,Manchester,/rc/clk?jk=6a54a5440e605728&fccid=bb4e3d67ccb0...
freq,4,23,21,356,998,1


### City 4: Birmingham

In [54]:
URLBIR = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=Birmingham&radius=30&jt=fulltime&limit=50&start={}"

In [55]:
bir_testdf = alljob_extractor(URLBIR, "Birmingham", max_pages=2500)

100%|██████████| 50/50 [04:34<00:00,  5.49s/it]


In [56]:
bir_testdf.head()

,Job_Title,Company,Salary,Location,City,Link
0,Data Scientist,Kainos,NaN,Birmingham,Birmingham,/rc/clk?jk=003ddc946281f5fc&fccid=538421af10c1...
1,Assistant Asset Engineer,Network Rail,"£25,806 - £35,000 a year",Birmingham B1 2ND,Birmingham,/rc/clk?jk=a58b50de6bfa2676&fccid=f5e16ad1713f...
2,DWP EO Work Coach – West Midlands Group – Mercia,UK Government - Department for Work and Pensions,"£27,565 a year",Redditch,Birmingham,/rc/clk?jk=9c5187a2b8e5d2e7&fccid=bde86870b61a...
3,National Data and Information Technician (cerv...,Public Health England,"£36,329 - £42,481 a year",Birmingham B3 2PW,Birmingham,/rc/clk?jk=dc1f2204f146c24f&fccid=56df6d0891c9...
4,Data Protection Analyst,Aldi,"£46,405 a year",Atherstone CV9 2SQ,Birmingham,/rc/clk?jk=3a06bcf786fe55d1&fccid=419801c05606...


In [57]:
bir_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,1000,1000,479,1000,1000,1000
unique,945,409,285,192,1,1000
top,Data Analyst,Balfour Beatty,"£24,907 - £30,615 a year",Birmingham,Birmingham,/rc/clk?jk=c3b7bc97c4e880a2&fccid=344344dda6e8...
freq,4,23,22,403,1000,1


### City 5: Newcastle

In [58]:
URLNEWC = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=Newcastle&radius=30&jt=fulltime&limit=50&start={}"

In [59]:
newc_testdf = alljob_extractor(URLNEWC, "Newcastle", max_pages=2500)

100%|██████████| 50/50 [04:29<00:00,  5.39s/it]


In [60]:
newc_testdf.head()

,Job_Title,Company,Salary,Location,City,Link
0,86794 - Senior Data Scientist,UK Government - Department for Work and Pensions,"£49,171 - £72,771 a year",Newcastle upon Tyne,Newcastle,/rc/clk?jk=3e30606881e8c53f&fccid=bde86870b61a...
1,Junior Data Warehouse ETL Developer,NHS Business Services Authority,"£24,907 - £30,615 a year",Newcastle upon Tyne,Newcastle,/rc/clk?jk=29fe64c4d2f5510d&fccid=7b42a43de4c4...
2,Executive Officer - Immigration and Protection...,UK Government - Home Office,"£24,883 - £31,519 a year",Newcastle upon Tyne,Newcastle,/rc/clk?jk=e393dc08d645c3ed&fccid=d6325847d65f...
3,Bio Processing Scientist,Leica Biosystems,NaN,Newcastle upon Tyne NE12 8EW,Newcastle,/rc/clk?jk=7291f6bd8a3c5179&fccid=f8d12c4ab90c...
4,COVID-19 Vaccination Programme Vaccinator,NHS Professionals,£10.09 - £11.19 an hour,Newcastle upon Tyne,Newcastle,/company/NHS-Professionals-(Bank)/jobs/Covid-V...


In [61]:
newc_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,1012,1012,652,1012,1012,1012
unique,920,503,368,228,1,1012
top,Staff Nurse,Newcastle University,"£31,365 - £37,890 a year",Newcastle upon Tyne,Newcastle,/company/Orchard-Care-Homes/jobs/Dementia-Supp...
freq,7,36,18,397,1012,1


### City 6: Bristol

In [62]:
URLBRIS = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=Bristol&radius=30&jt=fulltime&limit=50&start={}"

In [63]:
bris_testdf = alljob_extractor(URLBRIS, "Bristol", max_pages=2500)

100%|██████████| 50/50 [04:18<00:00,  5.17s/it]


In [64]:
bris_testdf.head()

,Job_Title,Company,Salary,Location,City,Link
0,Support Officer- Learning and Development,"UK Government - Department for Environment, Fo...","£29,157 - £32,160 a year",Bristol,Bristol,/rc/clk?jk=df3603c889a2574b&fccid=7d1f275fccbb...
1,Data Management Assistant,University of Bristol,"£19,133 - £21,236 a year",Bristol BS8 1QU,Bristol,/rc/clk?jk=f1587f9c56dd420c&fccid=3e2d0f19609a...
2,Student Records Administrator,University of Bristol,"£23,754 - £26,715 a year",Bristol BS8 1QU,Bristol,/rc/clk?jk=1e7f0de858321d21&fccid=3e2d0f19609a...
3,Development Scientist,NHS Blood and Transplant (NHSBT),"£31,365 - £37,890 a year",Bristol,Bristol,/rc/clk?jk=e0fc9367edc8dfd9&fccid=be7f1a7e5f11...
4,COVID-19 Vaccination Programme Vaccinator,NHS Professionals,£10.09 - £11.19 an hour,Bristol,Bristol,/company/NHS-Professionals/jobs/Covid-Vaccinat...


In [65]:
bris_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,1011,1011,484,1011,1011,1011
unique,954,384,288,164,1,1011
top,Software Engineer,UK Government - Ministry of Defence,"£40,000 a year",Bristol,Bristol,/rc/clk?jk=dce62baac5167631&fccid=e69c0a966f36...
freq,7,41,16,375,1011,1


### City 7: Leeds

In [66]:
URLLEED = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=Leeds&radius=30&jt=fulltime&limit=50&start={}"

In [67]:
leeds_testdf = alljob_extractor(URLLEED, "Leeds", max_pages=2500)

100%|██████████| 50/50 [04:17<00:00,  5.14s/it]


In [68]:
leeds_testdf.head()

,Job_Title,Company,Salary,Location,City,Link
0,Data Analyst Industrial Placement,HM Revenue and Customs,"£24,818 a year",Leeds,Leeds,/rc/clk?jk=a0e085f65822b513&fccid=a0bedfafc8a9...
1,Data Analyst,Mazars,NaN,Leeds,Leeds,/rc/clk?jk=85d8e6ff046e93e3&fccid=cf80abde47d3...
2,Data Analyst,Yorkshire Housing Limited,"£31,384 a year",Leeds,Leeds,/rc/clk?jk=0d5811e23b04c4e3&fccid=a16451726f4a...
3,Executive Education and Distance Learning Admi...,University of Bradford,"£20,130 - £21,814 a year",Bradford,Leeds,/rc/clk?jk=f5d5331c55cc4cdf&fccid=ebe1d34de054...
4,Manufacturing - Machine Operatives,WM Morrisons Supermarkets,NaN,Bradford,Leeds,/rc/clk?jk=6e00859599fea920&fccid=815d93b338ee...


In [69]:
leeds_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,1000,1000,560,1000,1000,1000
unique,940,398,320,159,1,1000
top,Business Analyst,Interface Recruitment,"£25,000 a year",Leeds,Leeds,/rc/clk?jk=1945f0a77f6ddf5b&fccid=6c46f9954c4d...
freq,6,63,19,344,1000,1


### City 8: Brighton

In [70]:
URLBRIG = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=Brighton&radius=30&jt=fulltime&limit=50&start={}"

In [71]:
brigh_testdf = alljob_extractor(URLBRIG, "Brighton", max_pages=2500)

100%|██████████| 50/50 [04:29<00:00,  5.39s/it]


In [72]:
brigh_testdf.head()

,Job_Title,Company,Salary,Location,City,Link
0,Public Health Data Analyst,East Sussex County Council,"£29,594 - £31,968 a year",Lewes,Brighton,/rc/clk?jk=803903688dc3065f&fccid=a16fa495e127...
1,Online Distance Learning Coordinator,University of Sussex,"£21,814 - £25,217 a year",Brighton,Brighton,/rc/clk?jk=6f62681517a428d3&fccid=5aa1e8076bd7...
2,Data Scientist - Remote,IQVIA,NaN,Brighton,Brighton,/rc/clk?jk=70fccbad7ec3e485&fccid=6b7a1dfe07e7...
3,Trainee Performance and Intelligence Analyst,Sussex NHS Commissioners,"£21,892 - £24,157 a year",Hove,Brighton,/rc/clk?jk=87b773d728e51b4e&fccid=bee895d04f2d...
4,Analytical Development Analyst,Custom Pharma Services,NaN,Brighton BN2,Brighton,/rc/clk?jk=de90e33937a44c27&fccid=3ad52c1e2844...


In [73]:
brigh_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,1017,1017,625,1017,1017,1017
unique,912,504,342,269,1,1017
top,Staff Nurse,SYK Recruitment Specialists,"£31,365 - £37,890 a year",Brighton,Brighton,/rc/clk?jk=52c5aa9baa7ff82d&fccid=d9caf355bb5b...
freq,9,17,24,169,1017,1


### City 9: Glasgow

In [74]:
URLGLW = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=Glasgow&radius=30&jt=fulltime&limit=50&start={}"

In [75]:
glw_testdf = alljob_extractor(URLGLW, "Glasgow", max_pages=2500)

100%|██████████| 50/50 [04:21<00:00,  5.23s/it]


In [76]:
glw_testdf.head()

,Job_Title,Company,Salary,Location,City,Link
0,Education Officer - Community Learning and Dev...,Scottish Government,"£48,930 - £61,006 a year",Glasgow,Glasgow,/rc/clk?jk=128bcdf56da7f7a4&fccid=7bbf7f016062...
1,Education Officer – Community Learning and Dev...,Scottish Government,"£48,930 - £61,006 a year",Glasgow,Glasgow,/rc/clk?jk=cda2a19d57c4c28a&fccid=7bbf7f016062...
2,Scientist 1,Merck KGaA,NaN,Glasgow G20 0XA,Glasgow,/rc/clk?jk=8f7961de43e543b9&fccid=68d0e0954a81...
3,Digital Engagement Officer -,Scottish Government,"£30,652 - £35,110 a year",Glasgow G2 8LU,Glasgow,/rc/clk?jk=c40e275f15abdd4e&fccid=7bbf7f016062...
4,Cash Management Analyst,AGS Airports,"£23,132 a year",Glasgow PA3,Glasgow,/rc/clk?jk=c3f53130b6a891f1&fccid=428cf1fa71b8...


In [77]:
glw_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,1013,1013,430,1013,1013,1013
unique,915,446,253,203,1,1013
top,Team Leader,"JPMorgan Chase Bank, N.A.","£35,000 a year",Glasgow,Glasgow,/company/GWG-Associates/jobs/Graduate-Data-Ana...
freq,7,47,11,506,1013,1


### City 10: Nottingham

In [78]:
URLNOTT = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=Nottingham&radius=30&jt=fulltime&limit=50&start={}"

In [79]:
nott_testdf = alljob_extractor(URLNOTT, "Nottingham", max_pages=2500)

100%|██████████| 50/50 [04:34<00:00,  5.50s/it]


In [80]:
nott_testdf.head()

,Job_Title,Company,Salary,Location,City,Link
0,Data Scientist,Boots,NaN,Nottingham,Nottingham,/rc/clk?jk=1a3d06dd529b03a6&fccid=72a97bc93221...
1,Data Analyst x 2 (Mon-Fri 5am-1pm and 7am-3pm),Page Personnel - UK,£11 - £12 an hour,Nottingham,Nottingham,/rc/clk?jk=3474311a07d0a048&fccid=e5ea1ff710ce...
2,Data Administrator,SF Recruitment,£10 - £11 an hour,Nottingham,Nottingham,/rc/clk?jk=1efd7e5815431a45&fccid=dbade4ef28af...
3,HO - Adjudicator's Office Investigator,HM Revenue and Customs,"£30,880 - £34,310 a year",Nottingham NG2,Nottingham,/rc/clk?jk=99847a16906fd37a&fccid=a0bedfafc8a9...
4,HO - Data Exploitation Analysts,HM Revenue and Customs,"£30,880 - £34,310 a year",Nottingham,Nottingham,/rc/clk?jk=0018e6d48ce2db7b&fccid=a0bedfafc8a9...


In [81]:
nott_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,1001,1001,508,1001,1001,1001
unique,957,346,297,200,1,1001
top,Registered Nurse,Nottingham University Hospitals NHS Trust,"£31,365 - £37,890 a year",Nottingham,Nottingham,/rc/clk?jk=3cb04d492cf05406&fccid=b85c5070c3d3...
freq,5,63,37,328,1001,1


### City 11: Belfast

In [82]:
URLBELF = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=Belfast&radius=30&jt=fulltime&limit=50&start={}"

In [83]:
belf_testdf = alljob_extractor(URLBELF, "Belfast", max_pages=2500)

100%|██████████| 50/50 [04:31<00:00,  5.42s/it]


In [84]:
belf_testdf.head()

,Job_Title,Company,Salary,Location,City,Link
0,Resourcing Assistant,The Open University UK,"£23,067 - £25,941 a year",Belfast,Belfast,/rc/clk?jk=1184afa5bb1ab8c7&fccid=f97e46d24243...
1,SAS Data Analyst,Celerion,NaN,Belfast,Belfast,/rc/clk?jk=e1d8aa8e78718c6f&fccid=b082f79c25a0...
2,Data Engineer,Kainos,NaN,Belfast,Belfast,/rc/clk?jk=02a529d99ca5f7f6&fccid=538421af10c1...
3,Markets KYC Controls Analyst (C09),Citi,NaN,Belfast,Belfast,/rc/clk?jk=7636223292e566a8&fccid=5bcd1ef0a7f4...
4,Senior Data Scientist,BazaarVoice,NaN,Belfast,Belfast,/rc/clk?jk=a1b1979c2ac3964a&fccid=e258616ae912...


In [85]:
belf_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,660,660,246,660,660,660
unique,596,256,153,103,1,660
top,Senior Software Engineer,Citi,"£25,000 - £30,000 a year",Belfast,Belfast,/rc/clk?jk=2949d44366b0b3a9&fccid=5bcd1ef0a7f4...
freq,8,44,10,408,660,1


### City 12: Cambridge

In [86]:
URLCAMB = "https://www.indeed.co.uk/jobs?q=(Data+or+Scientist+or+Data+or+Analyst+or+Data+or+Engineer+or+Machine+or+Learning+or+Engineer)+£20,000%2B&l=Cambridge&radius=30&jt=fulltime&limit=50&start={}"

In [87]:
camb_testdf = alljob_extractor(URLCAMB, "Cambridge", max_pages=2500)

100%|██████████| 50/50 [03:52<00:00,  4.65s/it]


In [88]:
camb_testdf.head()

,Job_Title,Company,Salary,Location,City,Link
0,DWP EO Work Coach – North and East Midlands Gr...,UK Government - Department for Work and Pensions,"£27,565 a year",Cambridge,Cambridge,/rc/clk?jk=a004f375c18b3f79&fccid=bde86870b61a...
1,Machine Learning Developer - Digital Healthcare,ElectronRx,NaN,Cambridge,Cambridge,/rc/clk?jk=087d96a718412dbe&fccid=cfc4bd3973b0...
2,Bioinformatics/ Data Analyst - 12 weeks intern...,Illumina,NaN,Cambridge,Cambridge,/rc/clk?jk=9289a33daa44a7cd&fccid=8524239b088a...
3,Biomedical Scientist,Nuffield Health,"£27,500 - £35,750 a year",Cambridge,Cambridge,/rc/clk?jk=b2dc50e800175366&fccid=3aca82fe3358...
4,Epic Application Analyst,Cambridge University Hospitals NHS Foundation ...,"£31,365 - £37,890 a year",Cambridge CB2 0QQ,Cambridge,/rc/clk?jk=146db142913ebb35&fccid=d68e40b451b3...


In [89]:
camb_testdf.describe()

,Job_Title,Company,Salary,Location,City,Link
count,1000,1000,408,1000,1000,1000
unique,948,321,262,165,1,1000
top,Clinical Psychologist,Abcam,"£24,907 - £30,615 a year",Cambridge,Cambridge,/rc/clk?jk=dedef3cdffdb1bcc&fccid=ece6beae9e41...
freq,4,45,20,357,1000,1


## Merge all cities to one df:

In [102]:
from functools import reduce

In [103]:
# all cols will match for all cities

cols = lon_testdf.columns
col_list = list(cols)
col_list

['Job_Title', 'Company', 'Salary', 'Location', 'City', 'Link']

In [106]:
# compile the list of dataframes you want to merge

uk_DF_lst = [lon_testdf, edi_testdf, mcr_testdf, 
             bir_testdf, newc_testdf, bris_testdf,
             leeds_testdf, brigh_testdf, glw_testdf,
             nott_testdf, belf_testdf, camb_testdf]

uk_df_full = reduce(lambda  left,right: pd.merge(left,right,on=col_list,
                                            how='outer'), uk_DF_lst)

In [107]:


uk_df_full.describe()

,Job_Title,Company,Salary,Location,City,Link
count,11734,11734,5718,11734,11734,11734
unique,8969,3521,2109,2211,12,11435
top,Business Analyst,IQVIA,"£24,907 - £30,615 a year",London,Edinburgh,/rc/clk?jk=a4fec4199c426f56&fccid=0c9b93ad0a8b...
freq,42,93,156,671,1022,2


## Deal with nulls in 'Salary'

In [108]:
# Assumption of around half removed
# nans only inserted in salary column

uk_df_full_minus_null_salary = uk_df_full.dropna(how='any')
uk_df_full_minus_null_salary.describe()

,Job_Title,Company,Salary,Location,City,Link
count,5718,5718,5718,5718,5718,5718
unique,4550,2082,2109,1588,12,5578
top,Staff Nurse,SYK Recruitment Specialists,"£24,907 - £30,615 a year",Newcastle upon Tyne,Newcastle,/rc/clk?jk=5accf7bfe7b9c997&fccid=22a9badd37b4...
freq,33,71,156,224,652,2


In [184]:
# rename
df = uk_df_full_minus_null_salary
df.head(1)


,Unnamed: 0,Job_Title,Company,Salary,Location,City,Link
0,0,Data and Research Analyst,Cancer Research UK,"£31,000 a year",Stratford,London,/rc/clk?jk=16e8b8e96810af24&fccid=0d06dfb448b2...


In [185]:
df = df.drop(columns='Unnamed: 0', axis=1)



In [186]:
df.describe()

,Job_Title,Company,Salary,Location,City,Link
count,5718,5718,5718,5718,5718,5718
unique,4550,2082,2109,1588,12,5578
top,Staff Nurse,SYK Recruitment Specialists,"£24,907 - £30,615 a year",Newcastle upon Tyne,Newcastle,/rc/clk?jk=5accf7bfe7b9c997&fccid=22a9badd37b4...
freq,33,71,156,224,652,2


In [187]:
df.isnull().sum()

Job_Title    0
Company      0
Salary       0
Location     0
City         0
Link         0
dtype: int64

In [188]:
df.groupby('City').describe()

Job_Title                                                    \
               count unique                                   top freq   
City                                                                     
Belfast          246    223                      Registered Nurse    4   
Birmingham       479    467                       Research Fellow    3   
Brighton         625    559                           Staff Nurse    9   
Bristol          484    472               IT Service Desk Analyst    2   
Cambridge        408    386                 Clinical Psychologist    4   
Edinburgh        447    400                           Staff Nurse    6   
Glasgow          430    398                           Team Leader    6   
Leeds            560    546  Children and Young People Key Worker    2   
London           377    354                      Business Analyst    6   
Manchester       502    478                       Project Manager    3   
Newcastle        652    597                           Staff Nurse    7   
Nottingham       508    488                      Registered Nurse    5   

           Company                                                          \
             count unique                                         top freq   
City                                                                         
Belfast        246    134                          Black Fox Solution   11   
Birmingham     479    222                       University of Warwick   18   
Brighton       625    353                 SYK Recruitment Specialists   17   
Bristol        484    205         UK Government - Ministry of Defence   41   
Cambridge      408    163             Wellcome Trust Sanger Institute   17   
Edinburgh      447    269                    Medicall Recruitment Ltd   15   
Glasgow        430    243                       University of Glasgow   30   
Leeds          560    230                       Interface Recruitment   57   
London         377    224                     Imperial College London   10   
Manchester     502    236  Manchester University NHS Foundation Trust   15   
Newcastle      652    333                        Newcastle University   36   
Nottingham     508    187   Nottingham University Hospitals NHS Trust   60   

           Salary                                       Location         \
            count unique                       top freq    count unique   
City                                                                      
Belfast       246    153  £25,000 - £30,000 a year   10      246     71   
Birmingham    479    285  £24,907 - £30,615 a year   22      479    137   
Brighton      625    342  £31,365 - £37,890 a year   24      625    223   
Bristol       484    288            £40,000 a year   16      484    108   
Cambridge     408    262  £24,907 - £30,615 a year   20      408    109   
Edinburgh     447    285  £25,000 - £30,000 a year    9      447    145   
Glasgow       430    253            £35,000 a year   11      430    143   
Leeds         560    320            £25,000 a year   19      560    113   
London        377    269  £45,780 - £51,393 a year    7      377    138   
Manchester    502    276  £24,907 - £30,615 a year   21      502    147   
Newcastle     652    368  £31,365 - £37,890 a year   18      652    180   
Nottingham    508    297  £31,365 - £37,890 a year   37      508    133   

                                      Link         \
                            top freq count unique   
City                                                
Belfast                 Belfast  109   246    246   
Birmingham           Birmingham  161   479    479   
Brighton               Brighton   77   625    625   
Bristol                 Bristol  158   484    484   
Cambridge             Cambridge   86   408    408   
Edinburgh             Edinburgh  194   447    447   
Glasgow                 Glasgow  191   430    430   
Leeds                     Leeds  164   560    560   
London              

## Salary cleaning

In [189]:
# All string

df['Salary'].value_counts()

£24,907 - £30,615 a year    156
£31,365 - £37,890 a year    154
£38,890 - £44,503 a year     93
£19,737 - £21,142 a year     88
£40,000 a year               80
                           ... 
£9.00 - £14.50 an hour        1
£14 - £18 an hour             1
£34,000 - £42,000 a year      1
£48,000 - £49,000 a year      1
£37,398 - £44,798 a year      1
Name: Salary, Length: 2109, dtype: int64

In [190]:
len(df['Salary'])

5718

In [191]:
# still needs to convert tuple to int & average the lower/upper values:

def salary_cleaner(col):
    
    """Filters to annual salries only, almost cleans it,
     leaving a tuple of strings. """
    
    mask = col.str.contains("a year")
    col = col[mask]
    
    col = col.replace("""[^0-9\.-]""", '', regex=True)
    col = col.str.split('-')
    col = col.apply(lambda x: tuple(x) if isinstance(x, list) else x)
    
    # needs int conversion & averaged out - done below
   

    
    return col

In [192]:
def tuple_avg_out(row):
    
    """To be used below after salary cleaner. """
    
    lst = [int(item) for item in row]
    return lst

In [193]:
# To check, there are only 4916 salaries listed (dropped from 5718) (annual only) over 12 uk cities - enough?

# still str in tuple, some only single values

df["Salary"] = salary_cleaner(df['Salary'])
df["Salary"]

0             (31000,)
1       (24907, 30615)
2             (27000,)
3             (32146,)
4       (28966, 37503)
             ...      
5713          (31000,)
5714               NaN
5715               NaN
5716               NaN
5717    (24907, 29000)
Name: Salary, Length: 5718, dtype: object

In [194]:
# Check for nulls now that only annual salaries remained

df.isnull().sum()

Job_Title      0
Company        0
Salary       802
Location       0
City           0
Link           0
dtype: int64

In [195]:
df = df.dropna(how='any')

In [200]:
df.describe(include='all')

,Job_Title,Company,Salary,Location,City,Link
count,4916,4916,4916,4916,4916,4916
unique,3980,1832,1658,1423,12,4788
top,Staff Nurse,UK Government - Ministry of Defence,"(24907, 30615)",Newcastle upon Tyne,Brighton,/rc/clk?jk=2f0e96760aaa8693&fccid=15af7d1916a5...
freq,21,65,156,184,529,2


In [196]:
df["Salary"][0]

('31000',)

In [197]:
df["Salary"][1]

('24907', '30615')

In [198]:
np.mean(tuple_avg_out(df["Salary"][1]))

27761.0

In [201]:
lst = []
for row in df["Salary"]:
    lst.append(np.mean(tuple_avg_out(row)))

In [202]:
pd.Series(lst).describe()

count      4916.000000
mean      34949.565094
std       14091.435163
min       20000.000000
25%       25230.500000
50%       32152.250000
75%       40000.000000
max      250000.000000
dtype: float64

In [203]:
df['Salary_cln'] = pd.Series(lst)

In [204]:
df.head()

,Job_Title,Company,Salary,Location,City,Link,Salary_cln
0,Data and Research Analyst,Cancer Research UK,"(31000,)",Stratford,London,/rc/clk?jk=16e8b8e96810af24&fccid=0d06dfb448b2...,31000.0
1,Data Scientist,NHS Midlands and Lancashire Commissioning Supp...,"(24907, 30615)",London SE1 6LH,London,/rc/clk?jk=195753e0562ed694&fccid=37c85e352ebb...,27761.0
2,Data Analyst – Content & Media,TalentPool,"(27000,)",London EC1R 4RB,London,/rc/clk?jk=8c1cf0cc8aa52ba9&fccid=3e983d79737f...,27000.0
3,Data Analyst,Central London Community Healthcare NHS Trust,"(32146,)",London SW1P 2PF,London,/rc/clk?jk=1d3ad812d7a09a77&fccid=7db6814b829e...,32146.0
4,NHS Test and Trace - Data Privacy Analyst,UK Government - Department of Health and Socia...,"(28966, 37503)",London,London,/rc/clk?jk=8a28e464e49cdce5&fccid=8b21fd09496a...,33234.5


In [205]:
# Drop old salary col:

df = df.drop(axis=1, columns='Salary')
df.head()

,Job_Title,Company,Location,City,Link,Salary_cln
0,Data and Research Analyst,Cancer Research UK,Stratford,London,/rc/clk?jk=16e8b8e96810af24&fccid=0d06dfb448b2...,31000.0
1,Data Scientist,NHS Midlands and Lancashire Commissioning Supp...,London SE1 6LH,London,/rc/clk?jk=195753e0562ed694&fccid=37c85e352ebb...,27761.0
2,Data Analyst – Content & Media,TalentPool,London EC1R 4RB,London,/rc/clk?jk=8c1cf0cc8aa52ba9&fccid=3e983d79737f...,27000.0
3,Data Analyst,Central London Community Healthcare NHS Trust,London SW1P 2PF,London,/rc/clk?jk=1d3ad812d7a09a77&fccid=7db6814b829e...,32146.0
4,NHS Test and Trace - Data Privacy Analyst,UK Government - Department of Health and Socia...,London,London,/rc/clk?jk=8a28e464e49cdce5&fccid=8b21fd09496a...,33234.5


In [206]:
# drop non annual salaried rows
df = df.dropna(how='any')
df.describe(include='all')

,Job_Title,Company,Location,City,Link,Salary_cln
count,4278,4278,4278,4278,4278,4278.000000
unique,3506,1639,1258,10,4153,NaN
top,Staff Nurse,UK Government - Ministry of Defence,Newcastle upon Tyne,Brighton,/rc/clk?jk=2f0e96760aaa8693&fccid=15af7d1916a5...,NaN
freq,18,61,184,529,2,NaN
mean,NaN,NaN,NaN,NaN,NaN,34890.379383
std,NaN,NaN,NaN,NaN,NaN,14105.751771
min,NaN,NaN,NaN,NaN,NaN,20000.000000
25%,NaN,NaN,NaN,NaN,NaN,25230.500000
50%,NaN,NaN,NaN,NaN,NaN,32070.750000
75%,NaN,NaN,NaN,NaN,NaN,40000.000000


In [207]:
df["Salary_cln"].value_counts()

27761.0     141
34627.5     130
35000.0     118
40000.0     117
25000.0      99
           ... 
29197.5       1
140000.0      1
43212.0       1
24024.0       1
21348.0       1
Name: Salary_cln, Length: 1245, dtype: int64

In [208]:
df.isnull().sum()

Job_Title     0
Company       0
Location      0
City          0
Link          0
Salary_cln    0
dtype: int64

### Export to a CSV file

In [209]:
# Cleaned df
df.to_csv('UK_jobs_cln.csv')